In [5]:
# 0. Load libraries #

import numpy as np
import pandas as pd
import os, time, warnings, random, gc, pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 200)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore')

# Load custom pre-processing functions:

def draw_histograms(df, variables, n_rows, n_cols):
    # stolen from https://stackoverflow.com/questions/29530355/plotting-multiple-histograms-in-grid
    fig=plt.figure()
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        df[var_name].hist(bins=10,ax=ax)
        ax.set_title(var_name+" Distribution")
    fig.tight_layout()  
    plt.show()


def fillna_mp_i1(df_train, df_test, df_pred, num_features, cat_features, num_fill='median', cat_fill='mode'):
    """This function speeds up filling missing values for 3 main datasets using different imputation methods.
    Later may replace it with some subclass.
    Example: fillna_mp_i1(X_train, X_test, X_pred, num_cols, cat_cols)"""
    # set df_pred to None if it does not exist
    if not ((cat_fill=='mode') and (num_fill=='median')):
        print ('Imputation method not Implemented yet!')
        return None
    
    df_train[num_features] = df_train[num_features].fillna(value=df_train[num_features].median())
    df_test[num_features] = df_test[num_features].fillna(value=df_train[num_features].median())
    df_train[cat_features] = df_train[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    df_test[cat_features] = df_test[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    if (df_pred is not None):
        df_pred[num_features] = df_pred[num_features].fillna(value=df_train[num_features].median())
        df_pred[cat_features] = df_pred[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    df_train[num_features+cat_features].count
    
    all_good = (
    (np.prod(df_train[num_features+cat_features].shape)==df_train[num_features+cat_features].count().sum()) and 
    (np.prod(df_test[num_features+cat_features].shape) == df_test[num_features+cat_features].count().sum()) and 
    (np.prod(df_pred[num_features+cat_features].shape) == df_pred[num_features+cat_features].count().sum()))
    if (all_good):
        print('Missing values imputed successfully')
    else:
        print('There are still some missing values...')
    
def add_misDummy_mp_i1(df_train, df_test, df_pred, features):
    """This function creates new dummy columns for missing features.
    Example: add_misDummy_mp_i1(X_train, X_test, X_pred, ['Age'])"""
    # set df_pred to None if it does not exist
    for feature_name in features:
        misColName = 'mis'+feature_name
        df_train.loc[df_train[feature_name].isnull(), misColName]=1
        df_train.loc[df_train[feature_name].notnull(), misColName]=0
        df_test.loc[df_test[feature_name].isnull(), misColName]=1
        df_test.loc[df_test[feature_name].notnull(), misColName]=0
        if (df_pred is not None):
            df_pred.loc[df_pred[feature_name].isnull(), misColName]=1
            df_pred.loc[df_pred[feature_name].notnull(), misColName]=0
   

def discretize_mp_i1(df_train, df_test, df_pred, feature, ntiles, delete_feature=False):
    """This function divides a continuous feature into quantile groups.
    Example: discretize_mp_i1(X_train, X_test, X_pred, 'Age', 15)"""
    # set df_pred to None if it does not exist
    _,bin = pd.qcut(df_train[feature], ntiles, retbins = True, labels = False, duplicates = 'drop')
    df_train[feature+'Ntile'] = pd.cut(df_train[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    df_test[feature+'Ntile'] = pd.cut(df_test[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (df_pred is not None):
        df_pred[feature+'Ntile'] = pd.cut(df_pred[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (delete_feature==True):
        df_train.drop(columns=[feature], inplace=True)
        df_test.drop(columns=[feature], inplace=True)
        df_pred.drop(columns=[feature], inplace=True)
    print('Discretized ',feature, ' into ', len(bin)-1, ' bins')


def log_transformer_mp_i1(df_train, df_test, df_pred, feature_subset=False, min_skew=3):
    """This function divides a continuous feature into quantile groups.
    Example: log_transformer_mp_i1(X_train, X_test, X_pred, feature_subset=num_cols)"""
    # set df_pred to None if it does not exist
    if (feature_subset==False):
        features_totransform = df_train.columns
    else:
        features_totransform = feature_subset.copy()
    skewed_vars = list(df_train.skew()[abs(df_train.skew())>min_skew].index)
    for col in list(set(skewed_vars)&set(features_totransform)):
        df_train[col] = np.log1p(df_train[col])
        df_test[col] = np.log1p(df_test[col])
        if (df_pred is not None):
            df_pred[col] = np.log1p(df_pred[col])
    print('Skewed columns log-transformed: ', list(set(skewed_vars)&set(features_totransform)))

In [15]:
with open('../input/amex-default-downsampled-01/amex_default_0.1sample.pickle', 'rb') as pickled_one:
    df = pickle.load(pickled_one)
display(df.head(),df.shape)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2017-05-30,0.937438,0.003936,0.003352,0.814304,0.002594,NaN,0.000545,0.003142,NaN,0.051788,0.006299,0.056203,0.294726,0.006012,0.004684,0.516281,0.463092,0.263574,NaN,0.048533,0.042849,0.006816,0.076369,0.004431,0.006775,0.005824,0.192826,0.577143,0.044728,NaN,0.003280,0.007864,1.000973,1.008453,0.000671,NaN,0.139550,0.003945,0.177336,0.169218,0.008859,0.009552,0.331046,NaN,0.005952,0.292875,0.002996,0.174586,0.005551,0.288992,0.081457,CO,O,0.009775,0.008577,NaN,0.645052,0.001056,1.0,0.001964,6.0,0.193990,0.000306,0.001171,0.000274,0.006808,0.005755,0.259391,0.015849,0.001570,0.507748,0.025547,NaN,0.001370,0.150138,0.137811,NaN,0.009684,0.004231,NaN,0.005078,0.003550,0.003895,0.000551,0.008775,NaN,0.003207,0.004396,0.004527,0.008564,0.006419,0.007386,NaN,0.000966,1.009854,0.058119,0.003344,0.007251,0.003969,0.008143,0.005722,0.009559,NaN,0.0,0.004709,0.001287,NaN,0.006435,0.008478,NaN,1,0.005740,0.005207,0.005587,0.000917,0.002636,0.001219,1.008942,0.000116,0.002600,0.001327,0.005468,0.006224,0.000029,0.003084,0.000054,0.005109,0.002801,0.301518,0.140075,0.082386,0.977059,0.001901,0.413177,1.000086,0.958687,0.661508,NaN,1.003331,0.006081,0.009483,NaN,1.005789,2.0,NaN,0.000268,NaN,NaN,NaN,1.009410,0.165166,NaN,0.001087,1.0,0.425165,0.0,4.0,0.418822,0.419459,0.0,0.547555,0.438120,0.003239,0.187171,0.006194,1.0,0.009093,1.005538,1.000016,0.003450,NaN,1.005729,0.001219,NaN,0.002394,0.004556,NaN,NaN,NaN,NaN,NaN,0.008103,0.006132,0.003267,NaN,0.008814,0.001211,0.001624
1,00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e...,2017-06-16,0.471242,0.001547,0.000233,0.816437,0.003437,NaN,0.005852,0.005877,NaN,0.208440,0.382538,0.313276,0.054264,0.002035,0.002264,0.951780,0.325470,0.740619,NaN,0.007790,1.047744,1.005298,0.076422,0.000206,0.005905,0.009041,0.148017,0.493458,0.039864,NaN,0.006881,0.003204,1.005171,1.001148,0.007656,NaN,0.010886,0.007330,0.448186,0.126476,0.004810,0.003998,0.619877,NaN,0.003674,0.375356,0.001085,0.599261,0.004723,0.285493,0.017829,CO,O,0.001949,0.005428,NaN,0.532836,0.001264,NaN,0.002074,5.0,0.184956,0.004232,0.004607,0.000368,0.002620,0.003322,1.008265,0.034615,0.001517,0.500245,0.995367,NaN,0.001463,0.365847,0.338917,NaN,0.007694,0.004741,NaN,0.004701,0.009776,0.004254,0.001066,0.003238,0.171472,0.003671,0.409818,0.001820,0.001104,0.008046,0.001091,NaN,0.001583,1.004534,0.211125,0.002501,0.007636,0.009532,0.004129,0.001284,0.008175,NaN,0.0,0.003752,0.004509,NaN,0.001339,0.008494,NaN,1,0.005238,0.008327,0.002736,0.007807,0.005138,0.009324,1.006314,0.005657,0.006198,0.006972,0.007000,0.001464,0.001995,0.004677,0.009206,0.008958,0.008415,0.302657,0.134319,0.086714,0.972725,0.000080,0.003580,0.008142,0.003662,NaN,NaN,0.008267,0.000910,0.002263,0.081330,0.015774,1.0,NaN,0.005904,NaN,NaN,NaN,1.002032,1.113697,NaN,0.004361,1.0,0.044666,0.0,4.0,0.045775,0.044385,0.0,0.755079,0.291698,0.009170,0.455286,0.006972,1.0,0.008171,1.004957,1.009854,0.000533,NaN,1.002896,0.006603,NaN,0.007623,0.006064,NaN,NaN,NaN,NaN,NaN

(54849, 190)

In [17]:
df_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')
display(len(df_labels))
df = pd.merge(df, df_labels, on='customer_ID', how='left')
display(df.shape, df.head())

458913

(54849, 191)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2017-05-30,0.937438,0.003936,0.003352,0.814304,0.002594,NaN,0.000545,0.003142,NaN,0.051788,0.006299,0.056203,0.294726,0.006012,0.004684,0.516281,0.463092,0.263574,NaN,0.048533,0.042849,0.006816,0.076369,0.004431,0.006775,0.005824,0.192826,0.577143,0.044728,NaN,0.003280,0.007864,1.000973,1.008453,0.000671,NaN,0.139550,0.003945,0.177336,0.169218,0.008859,0.009552,0.331046,NaN,0.005952,0.292875,0.002996,0.174586,0.005551,0.288992,0.081457,CO,O,0.009775,0.008577,NaN,0.645052,0.001056,1.0,0.001964,6.0,0.193990,0.000306,0.001171,0.000274,0.006808,0.005755,0.259391,0.015849,0.001570,0.507748,0.025547,NaN,0.001370,0.150138,0.137811,NaN,0.009684,0.004231,NaN,0.005078,0.003550,0.003895,0.000551,0.008775,NaN,0.003207,0.004396,0.004527,0.008564,0.006419,0.007386,NaN,0.000966,1.009854,0.058119,0.003344,0.007251,0.003969,0.008143,0.005722,0.009559,NaN,0.0,0.004709,0.001287,NaN,0.006435,0.008478,NaN,1,0.005740,0.005207,0.005587,0.000917,0.002636,0.001219,1.008942,0.000116,0.002600,0.001327,0.005468,0.006224,0.000029,0.003084,0.000054,0.005109,0.002801,0.301518,0.140075,0.082386,0.977059,0.001901,0.413177,1.000086,0.958687,0.661508,NaN,1.003331,0.006081,0.009483,NaN,1.005789,2.0,NaN,0.000268,NaN,NaN,NaN,1.009410,0.165166,NaN,0.001087,1.0,0.425165,0.0,4.0,0.418822,0.419459,0.0,0.547555,0.438120,0.003239,0.187171,0.006194,1.0,0.009093,1.005538,1.000016,0.003450,NaN,1.005729,0.001219,NaN,0.002394,0.004556,NaN,NaN,NaN,NaN,NaN,0.008103,0.006132,0.003267,NaN,0.008814,0.001211,0.001624,0
1,00013181a0c5fc8f1ea38cd2b90fe8ad2fa8cad9d9f13e...,2017-06-16,0.471242,0.001547,0.000233,0.816437,0.003437,NaN,0.005852,0.005877,NaN,0.208440,0.382538,0.313276,0.054264,0.002035,0.002264,0.951780,0.325470,0.740619,NaN,0.007790,1.047744,1.005298,0.076422,0.000206,0.005905,0.009041,0.148017,0.493458,0.039864,NaN,0.006881,0.003204,1.005171,1.001148,0.007656,NaN,0.010886,0.007330,0.448186,0.126476,0.004810,0.003998,0.619877,NaN,0.003674,0.375356,0.001085,0.599261,0.004723,0.285493,0.017829,CO,O,0.001949,0.005428,NaN,0.532836,0.001264,NaN,0.002074,5.0,0.184956,0.004232,0.004607,0.000368,0.002620,0.003322,1.008265,0.034615,0.001517,0.500245,0.995367,NaN,0.001463,0.365847,0.338917,NaN,0.007694,0.004741,NaN,0.004701,0.009776,0.004254,0.001066,0.003238,0.171472,0.003671,0.409818,0.001820,0.001104,0.008046,0.001091,NaN,0.001583,1.004534,0.211125,0.002501,0.007636,0.009532,0.004129,0.001284,0.008175,NaN,0.0,0.003752,0.004509,NaN,0.001339,0.008494,NaN,1,0.005238,0.008327,0.002736,0.007807,0.005138,0.009324,1.006314,0.005657,0.006198,0.006972,0.007000,0.001464,0.001995,0.004677,0.009206,0.008958,0.008415,0.302657,0.134319,0.086714,0.972725,0.000080,0.003580,0.008142,0.003662,NaN,NaN,0.008267,0.000910,0.002263,0.081330,0.015774,1.0,NaN,0.005904,NaN,NaN,NaN,1.002032,1.113697,NaN,0.004361,1.0,0.044666,0.0,4.0,0.045775,0.044385,0.0,0.755079,0.291698,0.009170,0.455286,0.006972,1.0,0.008171,1.004957,1.009854,0.000533,NaN,1.002896,0.006603,NaN,0.007623,0.006064,NaN,NaN,Na

In [18]:
df.count()

customer_ID    54849
S_2            54849
P_2            54387
D_39           54849
B_1            54849
               ...  
D_142           9508
D_143          53851
D_144          54467
D_145          53851
target         54849
Length: 191, dtype: int64